# 1. Получение токена VK (OAuth авторизация)

In [ ]:
import requests
from urllib.parse import urlencode

# Конфигурация приложения ВКонтакте
CLIENT_ID = '***'         # Ваш ID приложения
CLIENT_SECRET = '***'     # Ваш секретный ключ
REDIRECT_URI = 'https://oauth.vk.com/blank.html'

# Ссылка для авторизации
auth_url = 'https://oauth.vk.com/authorize'
auth_params = {
    'client_id': CLIENT_ID,
    'display': 'page',
    'redirect_uri': REDIRECT_URI,
    'scope': 'wall,groups,newsfeed',
    'response_type': 'code',
    'v': '5.131'
}

print(f"Перейдите по ссылке для авторизации: {auth_url}?{urlencode(auth_params)}")

# 2. Получение access_token по коду авторизации

In [ ]:
auth_code = input('Введите код авторизации: ')

token_url = 'https://oauth.vk.com/access_token'
token_params = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'redirect_uri': REDIRECT_URI,
    'code': auth_code
}

response = requests.get(token_url, params=token_params)
data = response.json()

access_token = data.get('access_token')
if access_token:
    print('Access token:', access_token)
else:
    print('Ошибка получения access token:', data)

# 3. Установка и загрузка зависимостей + keywords.txt

In [ ]:
!pip install gdown chardet

import gdown
import chardet

# Скачивание файла с ключевыми словами с Google Диска
file_url = 'https://drive.google.com/uc?id=***'  # Укажите ID файла из ссылки
# (Мой диск: напротив keywords.txt 3 точки - Открыть доступ - Все, у кого есть ссылка - Копировать ссылку
# - Вставить в код все, что находится в ссылке вместо *** https://drive.google.com/file/d/***/view?usp=sharing)
file_name = 'keywords.txt'

def download_keywords_file(url, output):
    print("Загружаем файл ключевых слов...")
    gdown.download(url, output, quiet=False)
    print(f"Файл сохранён как {output}")

def load_keywords(file_name):
    print("Читаем ключевые слова из файла...")
    with open(file_name, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        print(f"Определённая кодировка файла: {encoding}")

    try:
        content = raw_data.decode(encoding)
        # Разделение по строкам
        keywords = [line.strip() for line in content.splitlines() if line.strip()]
        return keywords
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return []

    # Загрузка ключевых слов
download_keywords_file(file_url, file_name)
keywords = load_keywords(file_name)

Загружаем файл ключевых слов...


FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=***

but Gdown can't. Please check connections and permissions.

# 4. Подключение Google Drive и подготовка к сохранению

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

folder_path = "/content/drive/My Drive/TextScope"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

save_path = os.path.join(folder_path, "vk_data.csv")

# 5. Подготовка функций для запроса и сохранения данных

In [ ]:
import pandas as pd
from datetime import datetime
import csv

def search_posts_by_keyword(keyword, access_token, count=100):
    url = (
        f"https://api.vk.com/method/newsfeed.search?"
        f"q={keyword}&count={count}&sort=2&access_token={access_token}&v=5.131"
    )
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        if 'error' in data:
            print(f"Ошибка API VK: {data['error']['error_msg']}")
            return []
        return data.get('response', {}).get('items', [])
    except Exception as e:
        print(f"Ошибка при запросе: {e}")
        return []

def load_existing_data(filename):
    return pd.read_csv(filename, encoding="utf-8-sig") if os.path.exists(filename) else pd.DataFrame()

def save_to_csv(df, filename):
    if not df.empty:
        df.to_csv(filename, index=False, encoding="utf-8-sig", quoting=csv.QUOTE_ALL, quotechar='"')
        print(f"Данные сохранены в {filename}")

# 6. Основной цикл сбора постов по ключевым словам

In [ ]:
import time

posts_df = load_existing_data(save_path)
access_token = input("Введите ваш access_token: ").strip()

for keyword in keywords:
    print(f"\nПоиск по ключу: {keyword}")
    posts = search_posts_by_keyword(keyword, access_token)

    if not posts:
        print(f"Нет постов для '{keyword}'")
        continue

    print(f"Найдено постов: {len(posts)}")

    data = []
    for post in posts:
        post_data = {
            'keyword': keyword,
            'text': post.get('text', ''),
            'owner_id': post.get('owner_id', ''),
            'post_id': post.get('id', ''),
            'date': datetime.fromtimestamp(post.get('date', 0)),
            'link': f"https://vk.com/wall{post.get('owner_id', '')}_{post.get('id', '')}"
        }
        data.append(post_data)

    new_df = pd.DataFrame(data)
    posts_df = pd.concat([posts_df, new_df], ignore_index=True).drop_duplicates()
    save_to_csv(posts_df, save_path)

    time.sleep(10)